In [1]:
# Imports 
import sys
from py2neo import neo4j, Graph
import MySQLdb


In [2]:
# Create MySQL connection
user = "lcdebaca"
password = "DcMCyskH49CKFnvEjx5C"
database = "starmetrics"

# invoke the connect() function, passing parameters in variables.
db = MySQLdb.connect( user = user, passwd = password, db = database )

# output basic database connection info.
print( db )

cursor = db.cursor( MySQLdb.cursors.DictCursor )
# Print Curser
print(cursor)

<_mysql.connection open to 'localhost' at 1bb0460>


In [3]:
# Test SQL 


# The Following is a Query of the starmetrics database
# It collects the following variables for Purdue University between the 
#years 2008 - 2012:  
#From the employee Table: id, employeeid, uniqueawardnumber, employeeoccupation
#From the occupation Table: occupationalclassification 

sql_select = "SElECT employee.employeeid, employee.uniqueawardnumber, employee.occupation_orig, \
occupation.occupationalclassification \
FROM starmetrics.employee \
INNER JOIN starmetrics.occupation ON \
starmetrics.employee.occupation_orig = occupation.occupation_orig \
WHERE starmetrics.employee.university = 'purdue' AND starmetrics.employee.year BETWEEN 2008 AND 2012;"

result_count = cursor.execute(sql_select);

# Count the number of rows returned 
print("Found " + str(result_count) + " rows")

# Testing
one = cursor.fetchone()
print(one.keys())
print(one)


# Store the results of the Query in a list
results = cursor.fetchall()


Found 1027382 rows
['occupationalclassification', 'occupation_orig', 'employeeid', 'uniqueawardnumber']
{'occupationalclassification': 'Undergraduate', 'occupation_orig': 'UNDERGRAD', 'employeeid': '90009387', 'uniqueawardnumber': "'00.500 NONGrant"}


In [4]:
# Create connection to the Neo4jsdatabase using defaults
# note, I imported Graph when I imported py2neo
graph_db_test = Graph()
print graph_db_test



<Graph uri=u'http://localhost:7474/db/data/'>


In [5]:
# Use this line to clear all nodes and relationships from the graph
#(good for debuging) 
#graph_db_test.delete_all()

In [6]:
# First Things First,
# Below is an exaple of how to create a single node with the properties employeeid, uniqueidnumber, occupation_orig, 
#and occupation identification. 
# If we wanted to make a single node, this would be the syntax 
new_node = neo4j.Node("employee", employeeid = '90014540', occupation_orig = "FACULTY", occupationalclassification = "Faculty")

# Python uses a slightly different syntax to query the Neo4js database
#If we were using the Neo4js Turminal, this would be the Cypher Query we would type:
## CREATE (Employee1:Employee { id_num: "4", employeeid:'90014540', uniqueawardnumber:'00.070 03-C-NE-PU',occupation_orig:"FACULTY", occupationalclassification:"Faculty" })



In [7]:
# If you want to test Cypher query syntax, This is a good site to experiemtn with;
#Cypher playground: http://console.neo4j.org/?_ga=1.113286210.459216022.1444237641
# Warning, I would suggest refreshing this page often 

In [8]:
# Above I saved the results of the SQL Query into a list called results
# Below I iterate over that list, create properties based on the varibles in the SQL database,
# and finaly create an 'employee' node with those properties

for employee in results:
    id_num = employee['employeeid']
    occ_orig = employee['occupation_orig']
    occ_class = employee['occupationalclassification']
    award_num = employee['uniqueawardnumber']
    
######################################################
#THIS IS THE LABELING BUG
######################################################
    
# THIS IS THE SYNTAX FOR CREATING A NODE IN PY2NEO. 

#"Employee" SHOULD BE THE NODE LABLE, BUT 
#IN THE CELL BELOW YOU CAN SEE THAT THE NODES HAVE NO LABEL
    neo4j.Node("Employee", employeeid = id_num, occcupation_orig = occ_orig, \
                      occupationalclassification = occ_class)


# HERE IS ALTERNATIVE PEICE OF CODE THAT SHOULD WORK: THE ERROR I GET IS THIS:

#UNDERGRAD not defined (line 1, column 60 (offset: 59))
#"CREATE (n:Employee {employeeid: 90010374, occupation_orig: UNDERGRAD, occupationalclassification: Undergraduate});"

    #graph_db_test.cypher.execute("CREATE (n:Employee {employeeid: " + id_num + ", occupation_orig: " + occ_orig + ", occupationalclassification: " + occ_class + "});")
    


In [9]:
#node = neo4j.Node("Cat", employeeid = 000000, occcupation_orig = "Professional Feline", \
#                      occupationalclassification = "FALCALTY")
#graph_db_test.cypher.execute("MATCH (n) RETURN n;") 

In [10]:
# Jsut to prove that these employees were moved into the NEO databse, 
#lets query the database and collect 10 nodes

##############################################################
# IF THE NODES HAD LABELS, THEY WOULD APPEAR IN THIS LIST, BUT THEY DONT
################################################################

some_employees = graph_db_test.cypher.execute("MATCH (n) RETURN n LIMIT 10")

print some_employees 

  | n
--+---



In [ ]:
# Close Cursor
cursor.close()

# close connection
db.close()